# Analysis of synchronized freezing
# How to analye FreezeFrame video (Win10)
    2019/12/26 Bug fix: we need to install win64 ffmpeg
    
    1) Video analysis manually. Write start and stop time as cvs file.
    2) Read entire cvs file which is not listed master table.
    3) Analysis.
    
---
## Step 1. Export AVI file using FreezeFrame Viewer
    1) install FreezeFrameInstaller4104.zip
    2) in FreezeFrame Viewer
        Video/Compress Image Files, as AVI file using MJPEG Compressor

## Step 2. Conversion avi to mp4
### Rotate and compress all avi videos in the current directory
    Provide current directory

In [ ]:
# Ubuntu path
#path = '/home/alexei/ubg-tera05/WD_Passport/Alexei/synchrony/110919'
# Windows path
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419'
path = r'Z:\wataru\WD_Passport\Alexei\synchrony\2019_9_12\122419_wi'
print(path)

In [ ]:
import os
import ffmpeg

#path = os.getcwd()
#print(path)

os.chdir(path)
print(os.listdir(path))

for file in os.listdir(path):
    base = os.path.splitext(file)[0]
    extn = os.path.splitext(file)[1]
    if extn == '.avi':
        # !ffmpeg -i {file} -vf "transpose=2,transpose=2" -vcodec libx264 {base}.mp4
        fileIN = os.path.join(path,file)
        print('file = ', fileIN, end = ' ')
        stream = ffmpeg.input(fileIN)
        stream = ffmpeg.hflip(stream)
        stream = ffmpeg.vflip(stream)
        fileOUT = os.path.join(path,base) + '.mp4'
        print('---> output = ', fileOUT)
        stream = ffmpeg.output(stream, fileOUT, vcodec='libx264')
        ffmpeg.run(stream)

print('end')

In [ ]:
import ffmpeg
stream = ffmpeg.input('input.mp4')
stream = ffmpeg.hflip(stream)
stream = ffmpeg.output(stream, 'output.mp4')
ffmpeg.run(stream)

## Step 3. Video Analysis using VideoPlayer

In [1]:
import videoplay as vp

video = r'W:\videos_synchrony\20191012\50.mp4'
vp.video_cursor(video)

Video resolution: (236, 304, 3)
total frame number: 722
	Reading _50_track.csv
	Writing _50_track.csv


## Step 4. Experimental GUI
    Not ready to use
   

In [ ]:
import tkinter as tk

def cbc(id, tex):
    return lambda : callback(id, tex)

def callback(id, tex):
    s = 'At {} f is {}\n'.format(id, id**id/0.987)
    tex.insert(tk.END, s)
    tex.see(tk.END)             # Scroll if necessary

top = tk.Tk()
tex = tk.Text(master=top)
tex.pack(side=tk.RIGHT)
bop = tk.Frame()
bop.pack(side=tk.LEFT)
for k in range(1,10):
    tv = 'Say {}'.format(k)
    b = tk.Button(bop, text=tv, command=cbc(k, tex))
    b.pack()

tk.Button(bop, text='Exit', command=top.destroy).pack()
top.mainloop()